In [1]:
from copy import deepcopy
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.9.2
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [2]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                    batch_size=-1, as_supervised=True)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-10-14 08:33:32.524086: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [3]:
from flex.data import FlexDataObject

flex_data = FlexDataObject.from_tfds_dataset(train_data)

In [4]:
from flex.data import FlexDatasetConfig, FlexDataDistribution

config = FlexDatasetConfig(seed=0)
config.n_clients = 2
config.replacement = False # ensure that clients do not share any data
config.client_names = ['client1', 'client2'] # Optional
flex_dataset = FlexDataDistribution.from_config(cdata=flex_data, config=config)

In [5]:
from flex.data import FlexDataDistribution

flex_dataset = FlexDataDistribution.iid_distribution(flex_data, n_clients=2)

# Primitive Functions

In [6]:
from flex.pool.flex_primitives import initialize_server_model
from flex.pool.flex_primitives import deploy_global_model_to_clients
from flex.pool.flex_primitives import deploy_model_to_clients
from flex.pool.flex_primitives import collect_weights
from flex.pool.flex_primitives import aggregate_weights
from flex.pool.flex_primitives import evaluate_model
from flex.pool.flex_primitives import train

In [7]:
# Defining the model

def define_model(*args):
    model = "https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2"
    hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam',
                    loss=tf.losses.BinaryCrossentropy(from_logits=True),
                    metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])
    return model

In [8]:
from flex.pool import FlexPool

flex_pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=initialize_server_model, model=define_model, verbose=1, model_params=[])

Initializing server model.


In [9]:
clients = flex_pool.clients
server = flex_pool.servers
print(f"Server node is indentified by {server.actor_ids}")
print(f"Client nodes are identified by {clients.actor_ids}")

Server node is indentified by ['server']
Client nodes are identified by ['client_0', 'client_1']


In [10]:
server.map(deploy_model_to_clients, clients, verbose=1)

Initializing model at a client
INFO:tensorflow:Assets written to: ram://2f1e61e7-e777-4a5f-93fa-129491310e87/assets


INFO:tensorflow:Assets written to: ram://2f1e61e7-e777-4a5f-93fa-129491310e87/assets


INFO:tensorflow:Assets written to: ram://17b2d811-b023-45ed-8073-98a342e3e287/assets


INFO:tensorflow:Assets written to: ram://17b2d811-b023-45ed-8073-98a342e3e287/assets


[None]

In [11]:
clients.map(train, batch_size=512, epochs=2)

Epoch 1/2
25/25 [==============================] - 3s 88ms/step - loss: 0.6186 - accuracy: 0.6833
Epoch 2/2
25/25 [==============================] - 2s 75ms/step - loss: 0.4963 - accuracy: 0.7974
Epoch 1/2
25/25 [==============================] - 2s 81ms/step - loss: 0.6178 - accuracy: 0.6869
Epoch 2/2
25/25 [==============================] - 2s 78ms/step - loss: 0.4918 - accuracy: 0.8042


[None, None]

In [12]:
aggregator = flex_pool.aggregators


In [14]:
clients.map(collect_weights, aggregator)

[None, None]

In [15]:
def fed_avg(agg_model):
    return np.mean(np.array(agg_model), axis=0)

In [16]:
aggregator.map(aggregate_weights, verbose=1, func_aggregate=fed_avg)

Aggregating weights


/var/folders/9d/8sk2j5nd30v6mvx2vx61hp980000gn/T/ipykernel_87007/2831934502.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.mean(np.array(agg_model), axis=0)


[None]

In [17]:
server.map(deploy_global_model_to_clients, clients)

[None]

In [18]:
test_examples, test_labels = test_data

In [19]:
server.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

Evaluating model at server
782/782 [==============================] - 6s 8ms/step - loss: 0.4717 - accuracy: 0.8066
Results on test data: [0.4716694951057434, 0.8065600395202637]


[None]

In [20]:
clients.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

Evaluating model at client.
391/391 [==============================] - 3s 8ms/step - loss: 0.4329 - accuracy: 0.8458
Results at client on client's data: [0.4328932762145996, 0.8457600474357605]
782/782 [==============================] - 5s 7ms/step - loss: 0.4717 - accuracy: 0.8066
Results on test data: [0.4716694951057434, 0.8065600395202637]
Evaluating model at client.
391/391 [==============================] - 3s 8ms/step - loss: 0.4299 - accuracy: 0.8482
Results at client on client's data: [0.4298538565635681, 0.848240077495575]
782/782 [==============================] - 6s 7ms/step - loss: 0.4717 - accuracy: 0.8066
Results on test data: [0.4716694951057434, 0.8065600395202637]


[None, None]

# Putting it all together

In [21]:
def train_n_rounds(n_rounds, batch_size, epochs):  
    pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=initialize_server_model, model=define_model, verbose=1, model_params=[])
    pool.servers.map(deploy_model_to_clients, pool.clients, verbose=1)
    for i in range(n_rounds):
        print(f"\nRunning round: {i}\n")
        clients.map(train, batch_size=512, epochs=2)
        pool.clients.map(train, batch_size=batch_size, epochs=epochs, verbose=1)
        pool.clients.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)
        pool.clients.map(collect_weights, pool.aggregators, verbose=1)
        pool.aggregators.map(aggregate_weights, verbose=1, func_aggregate=fed_avg)
        pool.servers.map(deploy_global_model_to_clients, pool.clients, verbose=1)
        pool.servers.map(evaluate_model, test_examples=test_examples, test_labels=test_labels)

In [22]:
train_n_rounds(n_rounds=4, batch_size=512, epochs=10)

Initializing server model.
Initializing model at a client
INFO:tensorflow:Assets written to: ram://dbe806d5-0614-4991-bcb7-6d7af03f69a2/assets


INFO:tensorflow:Assets written to: ram://dbe806d5-0614-4991-bcb7-6d7af03f69a2/assets


INFO:tensorflow:Assets written to: ram://dd62fbc4-5c38-40e4-b780-81d6b6d5758a/assets


INFO:tensorflow:Assets written to: ram://dd62fbc4-5c38-40e4-b780-81d6b6d5758a/assets



Running round: 0

Epoch 1/2
25/25 [==============================] - 5s 134ms/step - loss: 0.3892 - accuracy: 0.8650
Epoch 2/2
25/25 [==============================] - 3s 112ms/step - loss: 0.2935 - accuracy: 0.9082
Epoch 1/2
25/25 [==============================] - 4s 128ms/step - loss: 0.3816 - accuracy: 0.8682
Epoch 2/2
25/25 [==============================] - 3s 112ms/step - loss: 0.2839 - accuracy: 0.9092
Training model at client.
Epoch 1/10
25/25 [==============================] - 4s 125ms/step - loss: 0.6895 - accuracy: 0.5669
Epoch 2/10
25/25 [==============================] - 2s 88ms/step - loss: 0.5643 - accuracy: 0.7835
Epoch 3/10
25/25 [==============================] - 2s 92ms/step - loss: 0.4320 - accuracy: 0.8504
Epoch 4/10
25/25 [==============================] - 2s 89ms/step - loss: 0.3118 - accuracy: 0.8984
Epoch 5/10
25/25 [==============================] - 2s 91ms/step - loss: 0.2248 - accuracy: 0.9311
Epoch 6/10
25/25 [==============================] - 2s 91ms/ste

/var/folders/9d/8sk2j5nd30v6mvx2vx61hp980000gn/T/ipykernel_87007/2831934502.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.mean(np.array(agg_model), axis=0)


Evaluating model at server
782/782 [==============================] - 7s 9ms/step - loss: 0.2940 - accuracy: 0.8787
Results on test data: [0.294033020734787, 0.8787200450897217]

Running round: 1

Epoch 1/2
25/25 [==============================] - 3s 94ms/step - loss: 0.2214 - accuracy: 0.9334
Epoch 2/2
25/25 [==============================] - 2s 94ms/step - loss: 0.1674 - accuracy: 0.9524
Epoch 1/2
25/25 [==============================] - 3s 105ms/step - loss: 0.2107 - accuracy: 0.9374
Epoch 2/2
25/25 [==============================] - 3s 106ms/step - loss: 0.1572 - accuracy: 0.9574
Training model at client.
Epoch 1/10
25/25 [==============================] - 3s 97ms/step - loss: 0.0712 - accuracy: 0.9823
Epoch 2/10
25/25 [==============================] - 3s 102ms/step - loss: 0.0475 - accuracy: 0.9928
Epoch 3/10
25/25 [==============================] - 3s 104ms/step - loss: 0.0332 - accuracy: 0.9966
Epoch 4/10
25/25 [==============================] - 2s 94ms/step - loss: 0.0244 - ac

# END